In [15]:
import numpy as np
import pandas as pd

In [16]:
data = pd.read_csv('data/train.csv', encoding='Big5')
data = data.replace('NR', '0')
data = np.array(data)

In [17]:
hour = 9
feature_num = 18
day_per_month = 20
per_month_row = feature_num * day_per_month
total_month = int(len(data)/per_month_row) 

In [18]:
month_data = []
for i in range(total_month):
    l = data[i * per_month_row: i * per_month_row  + per_month_row]
    month_data.append(l)  

hour_data = []
for i in range(len(month_data)):
    tmp = []
    for j in range(len(month_data[i])):
        tmp.append(month_data[i][j])
    hour_data.append(tmp)

In [19]:
total = []
for i in range(len(hour_data)):
    df = pd.DataFrame(hour_data[i])
    row, col = df.shape
    tmp = None
    for j in range(day_per_month):
        per_day = df.iloc[j*feature_num:j*feature_num+feature_num,:]
        per_day = per_day.iloc[:,3:]
        
        if tmp is not None:
            tmp = pd.concat([tmp.reset_index(drop=True), per_day.reset_index(drop=True)], axis=1, ignore_index=True)
        else:
            tmp = pd.DataFrame(per_day)   
     
    total.append(tmp)

In [20]:
xss = []
yss = []
for i in range(len(total)):
    df = total[i]
    row, col = df.shape
    for j in range(col-hour-1):
        xs = df.iloc[:,j:j+hour]
        xs = xs.values.ravel()
        ys = float(df.iloc[9,j+hour+1])
        if(ys < 0):
            continue
        xss.append(xs)
        yss.append(float(ys))
        
xss = pd.DataFrame(xss).values.astype(np.float)
xss = np.column_stack(([1] * len(xss) ,xss))
yss = np.array(yss)

## feature scaling

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(xss)
print(scaler.mean_)
print(scaler.transform(xss))

[1.00000000e+00 2.25439155e+01 2.25525054e+01 2.25612742e+01
 2.25689692e+01 2.25745168e+01 2.25773801e+01 2.25770938e+01
 2.25751432e+01 2.25705619e+01 1.70259485e+00 1.70254116e+00
 1.70246958e+00 1.70234431e+00 1.70220115e+00 1.70205798e+00
 1.70202219e+00 1.70202219e+00 1.70193271e+00 3.90044739e-01
 3.90007158e-01 3.89992842e-01 3.90098425e-01 3.90173586e-01
 3.90273801e-01 3.90209377e-01 3.89858626e-01 3.89255548e-01
 1.40436650e-01 1.40572656e-01 1.40662133e-01 1.40918039e-01
 1.41215104e-01 1.41422691e-01 1.41512169e-01 1.41444166e-01
 1.41129205e-01 2.15300644e+00 2.15617394e+00 2.15952040e+00
 2.16197208e+00 2.16130995e+00 2.16116679e+00 2.16000358e+00
 2.15712241e+00 2.15059055e+00 1.01210093e+01 1.01311203e+01
 1.01416428e+01 1.01568361e+01 1.01657122e+01 1.01774517e+01
 1.01834646e+01 1.01837330e+01 1.01741410e+01 1.22596815e+01
 1.22724409e+01 1.22868468e+01 1.23045276e+01 1.23132427e+01
 1.23245347e+01 1.23301360e+01 1.23269506e+01 1.23111489e+01
 3.18946135e+01 3.193412

In [22]:
from IPython.display import clear_output
max_iter = 10 ** 4
epochs = 10 ** 3
lr = 1
w = np.zeros(feature_num * hour + 1)
w_lr = np.zeros(feature_num * hour+1)

for t in range(epochs):
    w_grad = None
    for m in range(max_iter):
        predict = np.dot(xss,w)
        w_grad = -(2 * np.dot(xss.T,(yss - predict)))
        w_lr = w_lr + w_grad ** 2
        w = w - lr/np.sqrt(w_lr) * w_grad

    clear_output()
    print(np.sqrt(np.mean([ x*x for x in (yss-predict)])))

8.046229995931112


KeyboardInterrupt: 

In [24]:
test = pd.read_csv('data/test.csv', encoding='Big5', header=None)
test = test.replace('NR', '0')

In [25]:
import pickle
with open('model/0225-scale.pkl', 'wb') as f:
    pickle.dump(w, f)
# with open('model/0223.pkl', 'rb') as f:
#       mynewlist = pickle.load(f)

In [34]:
ans = pd.read_csv('data/sampleSubmission.csv', encoding='Big5')
test_feature = 18
total_test = []
row, col = test.shape
test_number = int(row/test_feature)
print(test_number)
for i in range(test_number):
    df = test.iloc[i*test_feature: (i+1)*test_feature, 2:]
    xs = df.values.ravel().astype(np.float)
    xs = np.concatenate(([1], xs))
    xs = (scaler.transform([xs]))
    val = np.dot(xs,w)
    ans.iloc[i,1] = val
ans.to_csv('data/0225-scale.csv',index=False)

240


In [168]:
with open('model/0223-4.pkl', 'rb') as f:
      w = pickle.load(f)
        
predict = np.dot(xss,w)
error = predict - yss
# index = []
for i in range(len(error)):
    if abs(error[i]) <= 20:
        index.append(i)
index = set(index)
# print(error)
print(len(yss)-len(index))

140


In [169]:
test = pd.DataFrame(xss)
xss_o = test[test.index.isin(index)]
test = pd.DataFrame(yss)
yss_o = test[test.index.isin(index)]

In [170]:
from IPython.display import clear_output
max_iter = 10 ** 4
epochs = 10 ** 3
lr = 1
w = np.zeros(feature_num * hour + 1)
w_lr = np.zeros(feature_num * hour+1)
for t in range(epochs):
    w_grad = None
    for m in range(max_iter):
        predict = np.dot(xss,w)
        w_grad = -(2 * np.dot(xss.T,(yss - predict)))
        w_lr = w_lr + w_grad ** 2
        w = w - lr/np.sqrt(w_lr) * w_grad

    clear_output()
    print(np.sqrt(np.mean([ x*x for x in (yss-predict)])))

8.13433748307653


KeyboardInterrupt: 

In [30]:
myscale(xss)

[[ 0.         -1.3548871  -1.35576317 ...  0.26092589  0.25971901
  -1.14580047]
 [ 0.         -1.3548871  -1.35576317 ...  0.26092589 -1.14404986
  -1.33307513]
 [ 0.         -1.3548871  -1.51428551 ... -1.14393075 -1.33121905
  -0.8648885 ]
 ...
 [ 0.         -1.03772874 -0.88019615 ... -1.23758786 -0.58254231
  -0.30306454]
 [ 0.         -0.87914957 -0.72167381 ... -0.58198809 -0.30178854
  -0.39670187]
 [ 0.         -0.72057039 -0.56315147 ... -0.30101677 -0.39537313
  -0.11578989]]
